In [2]:
%cd data/

/mnt/data1/home/yoshida/my_research/data


In [3]:
test_data = "test_data"
train_data = "train_text"

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import pandas as pd
import gensim
from math import sqrt
import gensim.parsing
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from gensim.models import word2vec,LsiModel
import re
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import collections
from sklearn.model_selection import GridSearchCV
from func1 import *

/mnt/data1/home/yoshida/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
import MeCab

def doc2word_list(text_lis):
    tagger = MeCab.Tagger('-Ochasen')
    doc_lis = []

    for text in text_lis:
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            meta = node.feature.split(',')
            if meta[0] == '名詞' or meta[0] == "動詞" or meta[0] == "形容詞" or meta[0] == "副詞":
                if meta[6] == "*":
                    word_list.append(node.surface)
                else:
                    word_list.append(meta[6])
            node = node.next
        doc_lis.append(word_list)
        
    return doc_lis

In [21]:
def tag_get(name):
    #load data
    df = pd.read_csv(name + ".csv",encoding="utf-8",engine="python")
    text_lis = []
    tag_lis = []
    for text,tag in zip(df["text"],df["tag"]): #dfの対象名は随時変更する！
        text_lis.append(text)
        tag_lis.append(tag)
    return text_lis , tag_lis

text_lis, tag_lis = tag_get("oversample_text")
print(text_lis[:5])

['チャイナと北鮮に言ってやれ!核を放棄しろと!そして、お前らは非人道的過ぎると!', '科学検察は政治や軍部の下にいると科学的な検察が出来ないから独立性がないといけないんだけども、軍と政治以外で金の出所がない。', '「ここ第一で」、なるほどですね。|それなら思いきって書いちゃうことにしよう。||「オナペット」も昭和の言葉ですね。昭和エロ語辞典ができそうです。', '【極悪国策偏向反中反露:nhk】記録的高温のカナダ西部大規模な山火事発生し町の大部分焼失|★異常気象は利潤原理の資本主義のツケやが、それに扱いしよと思たら、既にそれは社会主義政策になっとるやろ。ダボス会議がグレートリセットと言い出しとるんが何よりの証拠や。', '民主主義社会においては"社会を構成する人員の最大公約数"への「最適化」が"公平"に一番近付くのに、活動家は自分(達)に合わせた「最適化」を求めるから"社会の解る"との間に齟齬が生じるのです|実態としては差別主義思想に基く"優遇"を求めているから非難されるのですよ|[定期]']


In [22]:
from gensim import corpora

# 名詞のリストになった記事群
t_lis = []
for text in text_lis:
    preprocessed = gensim.parsing.preprocess_string(text)
    pre = "".join(preprocessed)
    #print(pre)
    t_lis.append(pre)
    
#print(t_lis)
documents = doc2word_list(t_lis)
unfilter = doc2word_list(text_lis)

#print(documents)
dic = corpora.Dictionary(documents)
# 「出現頻度が3未満の単語」と「60%以上の文書で出現する単語」を排除
unfiltered = dic.token2id.keys()
dic.filter_extremes(no_below = 3, no_above = 0.6)
filtered = dic.token2id.keys()
filtered_out = set(unfiltered) - set(filtered)
bow_corpus = [dic.doc2bow(d) for d in documents]
# 辞書の保存(未知文書分類のため)
dic.save_as_text('a.txt')

In [23]:
print(len(bow_corpus))
print(len(set(unfiltered)), len(set(filtered)))
print(len(filtered_out))
print(documents[:3])
print(bow_corpus[2])

3000
9797 4345
5452
[['チャイナ', '北', '鮮', '言う', 'やる', '核', '放棄', 'する', 'お前', 'ら', '人道的', '過ぎる'], ['科学', '検察', '政治', '軍部', '下', 'いる', '科学', '的', '検察', '出来る', '独立', '性', 'ない', 'いける', 'ん', '軍', '政治', '以外', '金', '出所', 'ない'], ['ここ', '一', 'なる', 'それ', '思いきる', '書く', 'ちゃう', 'こと', 'する', 'オナペット', '昭和', '言葉', '昭和', 'エロ', '語', '辞典', 'できる', 'そう']]
[(1, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1)]


In [24]:
def vec2dense(vec, num_terms):
    '''Convert from sparse gensim format to dense list of numbers'''
    return list(gensim.matutils.corpus2dense([vec], num_terms=num_terms).T[0])

bow_docs = {}
bow_docs_all_zeros = {}
for i,text in enumerate(documents):
    sparse = dic.doc2bow(text)
    bow_docs[i] = sparse
    dense = vec2dense(sparse, num_terms=len(dic))
    #print(dense)
    bow_docs_all_zeros[i] = all(d == 0 for d in dense)

In [25]:
#print(bow_docs_all_zeros)
print(len(bow_docs))
print(bow_docs[503])

3000
[(1, 1), (13, 1), (23, 1), (46, 1), (151, 1), (193, 1), (210, 1), (235, 1), (262, 1), (611, 1), (654, 1), (722, 1), (770, 1), (883, 1), (1182, 1), (1817, 1), (2177, 1), (2178, 1)]


In [26]:
#LSI次元削除,num_topicの次元数に削減する
lsi_docs = {}
num_topics = 3
lsi_model = LsiModel(bow_docs.values(),
                           id2word=dic.load_from_text("a.txt"),
                           num_topics=num_topics)

for i in range(len(bow_docs)):
    vec = bow_docs[i]
    sparse = lsi_model[vec]
    dense = vec2dense(sparse, num_topics)
    lsi_docs[i] = sparse
    #print(dense)

In [27]:
#print(lsi_docs[1])

In [28]:
#Normalize a vector
unit_vecs = {}

for i in range(len(lsi_docs)):
    vec = vec2dense(lsi_docs[i],num_topics)
    norm = sqrt(sum(num**2 for num in vec))
    if norm == 0:
        norm = 1
    unit_vec = [num / norm for num in vec]
    unit_vecs[i] = unit_vec

#print(unit_vecs[1])

In [ ]:
vectorizer = TfidfVectorizer()
#text_lis,tag_lis = random_list(text_lis,tag_lis)

X_train = vectorizer.fit_transform(text_lis)
svd = TruncatedSVD(3)
X_train = svd.fit_transform(X_train)

# パラメータを dict 型で指定
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],  'gamma' : [0.001, 0.01, 0.1, 1, 10, 100]}

# validation set は GridSearchCV が自動で作成してくれるため，
# training set と test set の分割のみを実行すればよい

grid_search = GridSearchCV(SVC(), param_grid, cv=5)

# fit 関数を呼ぶことで交差検証とグリッドサーチがどちらも実行される
grid_search.fit(X_train, tag_lis)

In [29]:
train_data = [unit_vecs[i] for i in range(len(unit_vecs))]

In [30]:
# アンダーサンプリング
from imblearn.under_sampling import RandomUnderSampler

X_train, X_test, y_train, y_test = train_test_split(train_data, tag_lis, test_size=0.2)
target = "tag"
rs = RandomUnderSampler(random_state=42)
under_sampling ,label = rs.fit_resample(X_train, y_train)

clf = SVC(C=10,gamma=0.1, kernel="rbf")
clf.fit(under_sampling,label)
#print(under_sampling)
#print(y_train)

SVC(C=10, gamma=0.1)

In [37]:
pd_label = clf.predict(X_test)
print(len(pd_label))
#print(pd_label[:300])
#print(tag_lis[:300])

600


In [32]:
#predictionの準備
val_lis, valtag_lis = tag_get(test_data)
#print(val_lis[:5])
textval_lis = []
for text in val_lis:
    preprocessed = gensim.parsing.preprocess_string(text)
    pre = "".join(preprocessed)
    #print(pre)
    textval_lis.append(pre)
    
#print(t_lis)
docs = doc2word_list(textval_lis)
val_corpus = [dic.doc2bow(d) for d in docs]
val_docs = {}
bow_docs_all_zeros = {}
for i,text in enumerate(docs):
    sparse = dic.doc2bow(text)
    val_docs[i] = sparse
    dense = vec2dense(sparse, num_terms=len(dic))
    #print(dense)
    bow_docs_all_zeros[i] = all(d == 0 for d in dense)
    
#LSI次元削除,num_topicの次元数に削減する
lsi_valdocs = {}
num_topics = 3
lsi_model = LsiModel(val_docs.values(),
                           id2word=dic.load_from_text("a.txt"),
                           num_topics=num_topics)

for i in range(len(val_docs)):
    vec = val_docs[i]
    sparse = lsi_model[vec]
    dense = vec2dense(sparse, num_topics)
    lsi_valdocs[i] = sparse
    #print(dense)

#Normalize a vector
unit_valvecs = {}

for i in range(len(lsi_valdocs)):
    vec = vec2dense(lsi_valdocs[i],num_topics)
    norm = sqrt(sum(num**2 for num in vec))
    if norm == 0:
        norm = 1
    unit_vec = [num / norm for num in vec]
    unit_valvecs[i] = unit_vec
    
val_data = [unit_valvecs[i] for i in range(len(unit_valvecs))]

In [33]:
#predictionの実行
predict_label = clf.predict(val_data)
print(classification_report(valtag_lis, predict_label))

              precision    recall  f1-score   support

         0.0       0.58      0.39      0.47       213
         1.0       0.50      0.39      0.43       153
         2.0       0.13      0.53      0.21        34

    accuracy                           0.40       400
   macro avg       0.40      0.44      0.37       400
weighted avg       0.51      0.40      0.43       400



In [34]:
#TF-IDFとLSIによる次元削減による試み
"""
#Tf-Idfによる重要度重み付け
tfidf_model = models.TfidfModel(bow_docs.values())
tfidf_corpus = tfidf_model[bow_corpus]
#tf-idfモデルの保存(未知文書分類のため)
tfidf_model.save('tfidf_model.model')

#LSIによる次元削減
lsi_model = models.LsiModel(tfidf_corpus, id2word = dic, num_topics = 3)
lsi_corpus = lsi_model[tfidf_corpus]
#lsiモデルの保存(未知文書分類のため)
lsi_model.save('lsi_model.model')
"""

"\n#Tf-Idfによる重要度重み付け\ntfidf_model = models.TfidfModel(bow_docs.values())\ntfidf_corpus = tfidf_model[bow_corpus]\n#tf-idfモデルの保存(未知文書分類のため)\ntfidf_model.save('tfidf_model.model')\n\n#LSIによる次元削減\nlsi_model = models.LsiModel(tfidf_corpus, id2word = dic, num_topics = 3)\nlsi_corpus = lsi_model[tfidf_corpus]\n#lsiモデルの保存(未知文書分類のため)\nlsi_model.save('lsi_model.model')\n"

In [35]:
print(classification_report(y_test,pd_label))

              precision    recall  f1-score   support

           0       0.36      0.37      0.36       205
           1       0.41      0.51      0.46       174
           2       0.58      0.46      0.51       221

    accuracy                           0.44       600
   macro avg       0.45      0.45      0.44       600
weighted avg       0.46      0.44      0.45       600



In [36]:
##ここからTF-IDFを用いたベクトル化による実験処理

In [57]:
def random_list(x,y,seed=42):
    random.seed()
    p = list(zip(x, y))
    random.shuffle(p)
    x, y = zip(*p)
    return x,y

In [10]:
text_lis, tag_lis = tag_get("oversample_text")

vectorizer = TfidfVectorizer()
#text_lis,tag_lis = random_list(text_lis,tag_lis)

X_train = vectorizer.fit_transform(text_lis)
svd = TruncatedSVD(3)
X_train = svd.fit_transform(X_train)

from sklearn.model_selection import GridSearchCV

# パラメータを dict 型で指定
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],  'gamma' : [0.001, 0.01, 0.1, 1, 10, 100]}

# validation set は GridSearchCV が自動で作成してくれるため，
# training set と test set の分割のみを実行すればよい

grid_search = GridSearchCV(SVC(), param_grid, cv=5)

# fit 関数を呼ぶことで交差検証とグリッドサーチがどちらも実行される
grid_search.fit(X_train, tag_lis)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'gamma': [0.001, 0.01, 0.1, 1, 10, 100]})

In [11]:
l = collections.Counter(tag_lis)
print(l)

Counter({0: 1000, 1: 1000, 2: 1000})


In [12]:
X_test,y_test = tag_get("test_data")
X_test = vectorizer.fit_transform(X_test)
svd = TruncatedSVD(3)
X_test = svd.fit_transform(X_test)

pd_label = grid_search.predict(X_test)
print(pd_label)

print('Test set score: {}'.format(grid_search.score(X_test, y_test)))
print('Best parameters: {}'.format(grid_search.best_params_))
print('Best cross-validation: {}'.format(grid_search.best_score_))
print(classification_report(y_test,pd_label))

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2
 0 2 2 2 2 0 2 2 2 2 2 2 2 0 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 2 2 1 0 2 2
 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 1
 2 2 0 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 0 2 2 2 2
 2 2 2 2 2 2 2 2 2 1 2 2 2 2 0 2 1 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 2 0 2 0 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Test set score: 0.12
Best parameters: {'C': 100, 'gamma': 100}
Best cross-validation: 0.5366666666666667
              precision    recall  f1-score   support

         0.0       0.68     